In [ ]:
from pandas import Series, DataFrame
import config
import mysql.connector
from mysql.connector import errorcode

In [ ]:
#SQL credentials

sql_host = config.sql_host
sql_user = config.sql_user
sql_password = config.sql_password

In [ ]:
# retrieve data from sql select  --add functionality to select by time
def select(granularity):
    sql = ("SELECT * FROM {}".format(granularity))
    cnx = mysql.connector.connect(user=sql_user, password=sql_password,
                             host=sql_host, database='USDJPY')
    cursor = cnx.cursor()
    cursor.execute(sql)
    results = cursor.fetchall()
    cursor.close()
    cnx.close()
    return results

In [ ]:
# Create dataframe from sql query result
def dataframe(sqltable):
    names = ['time', 'openAsk', 'closeAsk', 'lowAsk', 'highAsk', 'openBid', 'closeBid', 'lowBid', 'highBid', 'volume']
    df = DataFrame(sqltable, columns = names)
    df.set_index('time', inplace=True)
    return df

In [ ]:
df = dataframe(select('H3'))

In [ ]:
df.head()

In [ ]:
#Change function to loop through stored data
def check_3hr_trend(self,time):
    self.params['count'] = 15
    self.params['end'] = datetime.timestamp(datetime.strptime(time,'%Y-%m-%d%H:%M:%S'))
    self.params['granularity'] = 'H3'

    req = requests.get(self.url, headers = self.headers, params = self.params)
    json_data = req.json()
    json_data = json_data['candles']
    time_reformatter(json_data)

    df = DataFrame(json_data)
    add_sma3_sma10(df)

    if df.ix[11, 'SMA10'] < df.ix[14, 'SMA10']:
        return True

In [ ]:
# Use for analysis
def add_sma3_sma10(df):
    df['SMA3'] = df['openBid'].rolling(window=3).mean()
    df['SMA10'] = df['openBid'].rolling(window=10).mean()

In [ ]:
# Use for analysis
def add_state(df):
    state = 'Above'
    for i in df.index:
        if df.ix[i, 'SMA3'] < df.ix[i, 'SMA10']:
            state = 'Below'
        else:
            state = 'Above'

        df.ix[i, 'State'] = state

In [ ]:
# Use for analysis
def rearrange_cols(df):
    return df[['time', 'openAsk', 'lowAsk', 'highAsk', 'closeAsk', 'openBid', 'lowBid', 'highBid',
             'closeBid', 'SMA3', 'SMA10', 'State',]]

In [ ]:
#Create function to check trend from SMA10? - Return Boolean - Work on DataFrame index
def check_trend(df, index):
    if df.ix[index-20, 'SMA10'] < df.ix[index, 'SMA10']:
        return True

In [ ]:
# change to work with json data

def crosses(df):
    crosses = 0

    for i in df.index[20:]:
        if check_trend(df, i):
            if df.ix[i-1,'State'] == 'Below' and df.ix[i, 'State'] == 'Above' : # True when SMA3 crosses over SMA10
                diff = ((df.ix[i, 'SMA10'] - df.ix[i-15, 'SMA10']) / df.ix[i, 'SMA10']) * 100
                spread = (df.ix[i, 'openAsk'] - df.ix[i, 'openBid']) *100
                print('{} at {} {}, Spread: {}'.format(
                        round(diff, 6), df.ix[i, 'time'],df.ix[i, 'date'], round(spread ,9)))
                crosses += 1

    print('Crossed above {} times.'.format(crosses))

In [ ]:
# change to work with json data

def crosses_with_3hr(dataframe):
    cross = 0
    for i in dataframe.index[20:]:
            if check_trend(dataframe, i):
                if USD_JPY.check_3hr_trend(dataframe.ix[i, 'date'] + dataframe.ix[i, 'time']):
                    if dataframe.ix[i-1,'State'] == 'Below' and dataframe.ix[i, 'State'] == 'Above' :
                        # True when SMA3 crosses over SMA10
                        spread = (dataframe.ix[i, 'openAsk'] - dataframe.ix[i, 'openBid']) *100
                        print('{} {}, Spread: {}'.format(
                                dataframe.ix[i, 'time'],dataframe.ix[i, 'date'], round(spread ,9)))
                        cross += 1

    print('Crossed above {} times.'.format(cross))